In [1]:
import plotly
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import simpleaudio as sa
from bisect import bisect_left, bisect_right

from plotly.subplots import make_subplots

Overview = pd.DataFrame()#columns =[
wave_obj = sa.WaveObject.from_wave_file("sound.wav") # sound that is played at end of calculations
empty = np.empty(0, dtype=int)

In [4]:
startfile  = 20
lastfile   = 20
fileprefix = 'D'
filesuffix = ''
savefileStr = '.new'

columnnames   = ['Healthy', 'Infected', 'Cured', 'Dead']
percentages   = [0.15, 0.50]# top always included
values        = [2, 2, 1] #how many days should be drawn at each percentage
savefile = False

for file in range(startfile,lastfile+1):
    data = pd.read_csv('./Datasets/'+fileprefix+str(file)+filesuffix+'.csv')
    index = file
    
    Overview.loc[index,'File'] = int(file)
    
    simulations = len(data['Day'][(data['Day'] == 1)])
    
    #-------Moments-------START----------------------#
    grouped      = data.groupby(['Day']).quantile(0.5)

    #interests = {}
    #interestdays = {}

    for i in [1,2]:
        maxinfected1 = grouped.max()['Infected'+str(i)]
        topinfected1 = grouped.idxmax()['Infected'+str(i)]
        interestdays = empty

        df1 = data[data['Day'] < topinfected1]
        df2 = data[data['Day'] > topinfected1]
        beforetop = df1.groupby(['Day']).quantile(0.5)
        aftertop  = df2.groupby(['Day']).quantile(0.5)
        aftertopreversed = aftertop.reindex(index=aftertop.index[::-1])
        moments = [beforetop, aftertopreversed]

        for moment, count in zip(moments,[1,2]):
            for p in percentages:
                foundday = find_neighbours(maxinfected1*p)
                interestdays = np.append(interestdays,foundday)            
        
        interestdays = np.append(interestdays,topinfected1)    
        interests = len(interestdays)    
        interestdays.sort()
        
        if i == 1: interestdays1 = interestdays
        if i == 2: interestdays2 = interestdays
    #-------Moments--------END------------------------#
    
    allpercentages = np.append(percentages, [1, percentages[1], percentages[0]])
    
    #-------Variance--------START------------------------#
    for nd in columnnames:        
        for day1, day2, daycount in zip(interestdays1, interestdays2, range(1,interests+1)):
            varrun1 = data.groupby(['Day']).var()[nd+'1'].iloc[int(day1)]
            varrun2 = data.groupby(['Day']).var()[nd+'2'].iloc[int(day2)]
            addvar = np.round(np.divide(np.subtract(varrun1,varrun2),varrun1),2)            
            Overview.loc[index,nd+str(daycount)] = addvar
    
    Overview.loc[index,'Interestdays Run1'] = ', '.join(map(str, interestdays1))
    Overview.loc[index,'Interestdays Run2'] = ', '.join(map(str, interestdays2))
    
    column = 'Infected'
    
    for i, interestdayarray in zip([1,2], [interestdays1, interestdays2]):
        totcount = 0
        
        for sim in range(1,simulations+1):
            momcount = 0
            for day, daycount in zip(interestdayarray, range(0,interests)):
                moment = data[(data['Simulations'] == sim) & (data['Day'] == day)]     #Column name is set to Simulations (mind the s)       
                if moment[column+str(i)].to_numpy() != 0: momcount = momcount+1 

            if momcount == 0: totcount = totcount+1

        Overview.loc[index,'No Pan. R'+str(i)] = totcount
    #-------Variance--------END------------------------#
    
    Overview.loc[index,'Percentages'] = ', '.join(map(str, allpercentages))
    Overview.loc[index,'Sims'] = round(simulations)
    Overview.loc[index,'Filename'] = fileprefix+str(file)+filesuffix

if savefile == True:
    if startfile == lastfile:
        Overview.to_csv('./ModelProcessedData/'+fileprefix+str(startfile)+savefileStr+'.DataFrame.csv', index=False)
    else: 
        Overview.to_csv('./ModelProcessedData/'+fileprefix+str(startfile)+'-'+str(lastfile)+savefileStr+'.DataFrame.csv', index=False)
    
play_obj = wave_obj.play()
#play_obj.wait_done()

Overview

,File,Healthy1,Healthy2,Healthy3,Healthy4,Healthy5,Infected1,Infected2,Infected3,Infected4,...,Dead3,Dead4,Dead5,Interestdays Run1,Interestdays Run2,No Pan. R1,No Pan. R2,Percentages,Sims,Filename
20,20.0,0.93,0.9,0.93,0.99,1.0,0.92,0.87,0.98,0.83,...,0.81,0.85,0.84,"46, 62, 82, 105, 121","41, 57, 75, 93, 106",41.0,0.0,"0.15, 0.5, 1.0, 0.5, 0.15",1000.0,D20


In [2]:
def find_neighbours(value):
  exactmatch=moment[moment['Infected'+str(i)]==value]
  if not exactmatch.empty:
      return exactmatch.index
  else:
      lowerneighbour_ind = moment[moment['Infected'+str(i)]<value]['Infected'+str(i)].idxmax()
      #upperneighbour_ind = moment[moment['Infected'+str(i)]>value]['Infected'+str(i)].idxmin()
      return [lowerneighbour_ind]#, upperneighbour_ind]